# Part-of-Speech Tagging - Thiago Pádua - 2020007066
A tarefa de Part-of-Speech Tagging (POS) consiste em rotular palavras de um texto de acordo com a sua classe gramatical, como substantivo, verbo, adjetivo. Esse processo é fundamental para diversas aplicações em Processamento de Linguagem Natural (PLN), pois permite uma compreensão mais detalhada da estrutura sintática e semântica das sentenças.

Por exemplo, considere a seguinte frase:
"A raposa azul dorme tranquilamente."

    "A" pode receber a tag de artigo definido.
    "raposa" pode receber a tag de substantivo.
    "azul" pode receber a tag de adjetivo.
    "dorme" pode receber a tag de verbo.
    "tranquilamente" pode receber a tag de advérbio.

Neste trabalho, o objetivo é explorar a tarefa de POS Tagging para a língua portuguesa, utilizando o corpus Mac-Morpho. Além disso, será implementado um modelo capaz de classificar palavras com precisão, analisando como o contexto influencia a atribuição de classes gramaticais. Por fim, investigaremos os desafios e as limitações do processo, discutindo as classes que apresentam maior e menor precisão ao longo do experimento.

## Carregamento dos Dados

In [3]:
import torch

In [4]:
test_file_path = "macmorpho-v3/macmorpho-test.txt"
train_file_path = "macmorpho-v3/macmorpho-train.txt"
dev_file_path = "macmorpho-v3/macmorpho-dev.txt"

def load_text_data(file_path):
    """
    :param file_path: path to the .txt file containing the data
    :return: List of sentences, where each sentence is a list of tuples (word, tag)
    """
    sentences = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line:  # Ignore empty lines
                # Generate a list of tuples (word, tag)
                word_tags = [tuple(word.split('_')) for word in line.split()]
                sentences.append(word_tags)
    return sentences

test_data = load_text_data(test_file_path)
train_data = load_text_data(train_file_path)
dev_data = load_text_data(dev_file_path)


In [5]:
for i, sentence in enumerate(test_data[:3]):
    print(f"Sentence {i+1}: {sentence}")

Sentence 1: [('Salto', 'N'), ('sete', 'ADJ')]
Sentence 2: [('O', 'ART'), ('grande', 'ADJ'), ('assunto', 'N'), ('da', 'PREP+ART'), ('semana', 'N'), ('em', 'PREP'), ('Nova', 'NPROP'), ('York', 'NPROP'), ('é', 'V'), ('a', 'ART'), ('edição', 'N'), ('da', 'PREP+ART'), ('revista', 'N'), ('"', 'PU'), ('New', 'NPROP'), ('Yorker', 'NPROP'), ('"', 'PU'), ('que', 'PRO-KS'), ('está', 'V'), ('nas', 'PREP+ART'), ('bancas', 'N'), ('.', 'PU')]
Sentence 3: [('Número', 'N'), ('duplo', 'ADJ'), ('especial', 'ADJ'), (',', 'PU'), ('é', 'V'), ('inteirinho', 'ADJ'), ('dedicado', 'PCP'), ('a', 'PREP'), ('ensaios', 'N'), ('sobre', 'PREP'), ('moda', 'N'), ('.', 'PU')]


## Metodologia

In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("lisaterumi/postagger-portuguese")
model = AutoModelForTokenClassification.from_pretrained("lisaterumi/postagger-portuguese")

/home/thiago/UFMG/9_periodo/NLP/TP2/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
teste = "A cidade mais linda de Minas Gerais é Belo Horizonte".split(sep=' ')

test_words = [t[0] for t in teste]
inputs = tokenizer(teste, return_tensors="pt", is_split_into_words=True, padding=True, truncation=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [9]:
print("Tokens gerados:", tokenizer.convert_ids_to_tokens(inputs["input_ids"][0]))

Tokens gerados: ['[CLS]', 'A', 'cidade', 'mais', 'lin', '##da', 'de', 'Minas', 'Gerais', 'é', 'Belo', 'Horizonte', '[SEP]']


In [10]:
# Obter os índices das palavras originais correspondentes a cada token
word_ids = inputs.word_ids(batch_index=0)  # Adicione o batch_index para evitar erros de dimensão

# Exibir os mapeamentos
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

# Mostrar os tokens e as palavras originais
for token, word_id in zip(tokens, word_ids):
    palavra_original = teste[word_id] if word_id is not None else None
    print(f"Token: {token}, Palavra Original: {palavra_original}")


Token: [CLS], Palavra Original: None
Token: A, Palavra Original: A
Token: cidade, Palavra Original: cidade
Token: mais, Palavra Original: mais
Token: lin, Palavra Original: linda
Token: ##da, Palavra Original: linda
Token: de, Palavra Original: de
Token: Minas, Palavra Original: Minas
Token: Gerais, Palavra Original: Gerais
Token: é, Palavra Original: é
Token: Belo, Palavra Original: Belo
Token: Horizonte, Palavra Original: Horizonte
Token: [SEP], Palavra Original: None


In [ ]:
outputs = model(**inputs)
logits = outputs.logits
predictions = torch.argmax(logits, dim=-1)

# Mapear previsões para palavras originais
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
word_ids = inputs.word_ids(batch_index=0)


tags = []
current_word = None
for token, word_id, pred in zip(tokens, word_ids, predictions[0].tolist()):
    if word_id is None:  # Ignore special tokens
        continue

    # Associate tag only with the first token of each word. Repeated ids will be ignored
    if word_id != current_word:
        tags.append(model.config.id2label[pred])
        current_word = word_id

print("Tags preditas para palavras:", list(zip(teste, tags)))

[None, 0, 1, 2, 3, 3, 4, 5, 6, 7, 8, 9, None]
Tags preditas para palavras: [('A', 'ART'), ('cidade', 'N'), ('mais', 'ADV'), ('linda', 'ADJ'), ('de', 'PREP'), ('Minas', 'NPROP'), ('Gerais', 'NPROP'), ('é', 'V'), ('Belo', 'NPROP'), ('Horizonte', 'NPROP')]


In [12]:
import torch

outputs = model(**inputs)
logits = outputs.logits
predictions = torch.argmax(logits, dim=-1)

# Mapear previsões para palavras originais
tags = [model.config.id2label[pred] for pred, word_id in zip(predictions[0].tolist(), word_ids) if word_id is not None]

print("Palavras originais:", test_words)
print("Tags preditas:", tags)


Palavras originais: ['A', 'c', 'm', 'l', 'd', 'M', 'G', 'é', 'B', 'H']
Tags preditas: ['ART', 'N', 'ADV', 'ADJ', 'ADJ', 'PREP', 'NPROP', 'NPROP', 'V', 'NPROP', 'NPROP']


In [13]:
filtered_tags = [tag for tag, word_id in zip(tags, word_ids) if word_id is not None]

print("Tags filtradas:", filtered_tags)

Tags filtradas: ['N', 'ADV', 'ADJ', 'ADJ', 'PREP', 'NPROP', 'NPROP', 'V', 'NPROP', 'NPROP']


In [14]:
import torch

predicted_token_class_ids = torch.argmax(logits, dim=-1)

In [15]:
label_map = model.config.id2label
predicted_labels = [label_map[label_id.item()] for label_id in predicted_token_class_ids[0]]

In [16]:
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
pos_tags = list(zip(tokens, predicted_labels))

In [17]:
for token, tag in pos_tags:
    print(f"{token}: {tag}")

[CLS]: <pad>
A: ART
cidade: N
mais: ADV
lin: ADJ
##da: ADJ
de: PREP
Minas: NPROP
Gerais: NPROP
é: V
Belo: NPROP
Horizonte: NPROP
[SEP]: <pad>
